<h1>AIM:</h1>
To implement real-time face detection using Haar Cascade classifiers and perform various image processing operations (negative transformation, image enhancement, and background blurring) on captured video frames.

<h1>OBJECTIVES:</h1>
<ol>
<li>Capture live video stream from a webcam</li>
<li>Detect faces using Haar Cascade classifier</li>
<li>Save extracted frames with face annotations</li>
<li>Generate negative (inverted) versions of frames</li>
<li>Enhance image quality through denoising, contrast adjustment, and sharpening</li>
<li>Implement background blurring while keeping faces in focus</li>
</ol>

<h1>Computer Vision Theory</h1>

## 1. Camera Basics

### Digital Camera Operation
- Converts light into electronic signals using CMOS/CCD sensors

### Frame Resolution
- Determined by pixel dimensions (e.g., 640×480 in this implementation)

### Frame Rate
- Number of frames captured per second (20 fps in this implementation)

### Color Space
- Uses BGR format in OpenCV (Blue-Green-Red channels)

---

## 2. Haar Cascade Classifiers

### Principle
- Machine learning-based approach for object detection using Haar-like features

### Training
- Requires positive (object) and negative (non-object) images

### Face Detection
- Uses edge and line features to identify facial patterns
- Implemented through cascade of simple classifiers (Viola-Jones algorithm)
- OpenCV provides pre-trained frontal face detection model

---

## 3. Image Processing Techniques

### Negative Transformation
- Pixel inversion using `cv2.bitwise_not()`

### Denoising
- Non-local Means Denoising preserves details while reducing noise

### Contrast Enhancement
- CLAHE (Contrast Limited Adaptive Histogram Equalization)
- Applied on luminance channel in LAB color space

### Sharpening
- Convolution with kernel to enhance edges

### Background Blurring
- Gaussian blur (σ=0, kernel size=51) for background
- Face masking using elliptical regions
- Alpha blending for smooth transitions

---

## 4. OpenCV Workflow

1. Video capture initialization (`cv2.VideoCapture`)
2. Frame-by-frame processing loop
3. Grayscale conversion for feature detection
4. Haar Cascade face detection (`detectMultiScale`)
5. Real-time display and file operations
6. Resource cleanup (`release()`, `destroyAllWindows()`)

In [ ]:
# %pip install openCV.python

In [1]:
import cv2
import os
import numpy as np

#Open default camera
cam = cv2.VideoCapture(0)
frame_width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("Frame Width", frame_width)
print("Frame Height", frame_height)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (frame_width, frame_height))

# Load the cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize the video capture
cam = cv2.VideoCapture(0)  # Use `0` for webcam or replace with video file path

# Create a directory to save the frames
output_dir = "extracted_frames"
os.makedirs(output_dir, exist_ok=True)

frame_count = 0  # To keep track of the frames

while True:
    ret, frame = cam.read()
    if not ret:
        break
    
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    
    # Draw rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    # Save the frame as an image
    frame_path = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")  # Save as frame_0001.jpg, frame_0002.jpg, etc.
    cv2.imwrite(frame_path, frame)
    frame_count += 1
    
    # Display the frame
    cv2.imshow('Camera', frame)
    
    # Break the loop
    if cv2.waitKey(1) == ord('q'):
        break

# Release the VideoCapture object
cam.release()
cv2.destroyAllWindows()

print(f"Frames saved in the folder: {output_dir}")

# while True:
#     ret, frame = cam.read()
#     out.write(frame)
#     cv2.imshow('Camera', frame)
    
#     if cv2.waitKey(1) == ord('q'):
#         break

# cam.release()
# out.release()
# cv2.destroyAllWindows()

Frame Width 640
Frame Height 480
Frames saved in the folder: extracted_frames


In [2]:
# Directories for input frames and output negatives
input_dir = "extracted_frames"
output_dir = "negative_frames"
os.makedirs(output_dir, exist_ok=True)

# Iterate through each frame in the input directory
for frame_name in os.listdir(input_dir):
    frame_path = os.path.join(input_dir, frame_name)
    
    # Read the frame
    frame = cv2.imread(frame_path)
    if frame is None:
        print(f"Could not read {frame_path}")
        continue
    
    # Convert the frame to its negative
    negative_frame = cv2.bitwise_not(frame)
    
    # Save the negative frame
    output_path = os.path.join(output_dir, f"negative_{frame_name}")
    cv2.imwrite(output_path, negative_frame)

print(f"Negative frames saved in the folder: {output_dir}")

Negative frames saved in the folder: negative_frames


In [3]:
# Directories for input frames and output enhanced images
input_dir = "extracted_frames"
output_dir = "enhanced_frames"
os.makedirs(output_dir, exist_ok=True)

def enhance_image(frame):
    """Apply enhancement pipeline to a frame"""
    # 1. Denoising
    denoised = cv2.fastNlMeansDenoisingColored(frame, None, 10, 10, 7, 21)
    
    # 2. Contrast Enhancement (CLAHE in LAB color space)
    lab = cv2.cvtColor(denoised, cv2.COLOR_BGR2LAB)
    l_channel, a_channel, b_channel = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_l = clahe.apply(l_channel)
    lab_enhanced = cv2.merge((enhanced_l, a_channel, b_channel))
    bgr_enhanced = cv2.cvtColor(lab_enhanced, cv2.COLOR_LAB2BGR)
    
    # 3. Sharpening
    kernel = np.array([[-1, -1, -1],
                       [-1,  9, -1],
                       [-1, -1, -1]])
    sharpened = cv2.filter2D(bgr_enhanced, -1, kernel)
    
    # 4. Brightness/Contrast Adjustment
    alpha = 1.2  # Contrast control (1.0-3.0)
    beta = 30    # Brightness control (0-100)
    final = cv2.convertScaleAbs(sharpened, alpha=alpha, beta=beta)
    
    return final

# Process all frames in input directory
for frame_name in os.listdir(input_dir):
    frame_path = os.path.join(input_dir, frame_name)
    
    # Read frame
    frame = cv2.imread(frame_path)
    if frame is None:
        print(f"Could not read {frame_path}")
        continue
    
    # Apply enhancement
    enhanced_frame = enhance_image(frame)
    
    # Save enhanced frame
    output_path = os.path.join(output_dir, f"enhanced_{frame_name}")
    cv2.imwrite(output_path, enhanced_frame)

print(f"Enhanced frames saved in: {output_dir}")

Enhanced frames saved in: enhanced_frames


In [4]:
# Directories for input frames and output blurred background images
input_dir = "extracted_frames"
output_dir = "blurred_bg_frames"
os.makedirs(output_dir, exist_ok=True)

# Load face cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def blur_background(frame):
    """Apply background blur while keeping faces in focus"""
    # Convert to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    
    # Create a mask for foreground (faces)
    mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    for (x, y, w, h) in faces:
        # Create elliptical mask for more natural shape
        center = (x + w//2, y + h//2)
        axes = (int(w*0.6), int(h*0.6))
        cv2.ellipse(mask, center, axes, 0, 0, 360, 255, -1)
    
    # Apply Gaussian blur to entire image
    blurred = cv2.GaussianBlur(frame, (51, 51), 0)
    
    # Combine original and blurred images using the mask
    result = np.where(mask[..., np.newaxis], frame, blurred)
    
    # Optional: Add subtle blur to mask edges for smoother transition
    mask_blur = cv2.GaussianBlur(mask, (15, 15), 0)[..., np.newaxis]/255.0
    result = (frame * mask_blur + blurred * (1 - mask_blur)).astype(np.uint8)
    
    return result

# Process all frames in input directory
for frame_name in os.listdir(input_dir):
    frame_path = os.path.join(input_dir, frame_name)
    
    # Read frame
    frame = cv2.imread(frame_path)
    if frame is None:
        print(f"Could not read {frame_path}")
        continue
    
    # Apply background blur
    blurred_frame = blur_background(frame)
    
    # Save result
    output_path = os.path.join(output_dir, f"blurred_bg_{frame_name}")
    cv2.imwrite(output_path, blurred_frame)

print(f"Blurred background frames saved in: {output_dir}")

Blurred background frames saved in: blurred_bg_frames


The frames extracted and the video can be found in my github:- 